### This Notebook Provides some Notes about using PyTorch for LinearRegression

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### `Building a Simple Linear Regression Model`（簡単な線形モデルの構築）

In [2]:
## loading and preparing the dataset
## using only 5 features and 1000 sample(800 for train and 200 for test)
# 5つの特徴量と1000サンプル(訓練用800サンプル、テスト用200サンプル)のみ使用

# make_regression関数を使用して、回帰問題のためのデータセットを生成しています。
# このデータセットは1000サンプルと5つの特徴量を持ち、ノイズを20追加しています。
data_reg = datasets.make_regression(n_samples=1000, n_features=5, noise=20, random_state=42)

X = data_reg[0]
y = data_reg[1]

# reshape y（yを列ベクトルにしてるんだ）
y = y.reshape(-1, 1)

## split the data（データの分割と標準化）
# データを訓練セットとテストセットに分割しています。
# 訓練セットには80%のデータ（800サンプル）、テストセットには20%のデータ（200サンプル）が含まれています。
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    shuffle=True, random_state=42)

## standardize the dataset（データセットの標準化）
# StandardScalerを使って特徴量の標準化を行っています。
# 訓練データでフィッティングし、そのスケールを使って訓練データとテストデータの標準化を行っています。
sacler = StandardScaler()
X_train_scaled = sacler.fit_transform(X_train)
X_test_scaled = sacler.transform(X_test)

## convert to float32 then convert to tensors
# データをfloat32に変換し、NumPy配列をPyTorchのテンソルに変換しています。
# これにより、PyTorchのニューラルネットワークにデータを入力できるようになります。
X_train_scaled = torch.from_numpy(X_train_scaled.astype(np.float32))
X_test_scaled = torch.from_numpy(X_test_scaled.astype(np.float32))

y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

#### Build the Model

In [3]:
## build the Model（モデルの定義）
input_size = X_train_scaled.shape[1] #入力の特徴量の数
output_size = 1   ## taregt（1次元の出力）

class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim): #initはクラスのコンストラクタ、クラスがインスタンス化された時に実行
        super(LinearRegression, self).__init__() # nn.Moduleのコンストラクタを呼び出している、基底クラスの初期化
        self.lin_reg = nn.Linear(in_features=input_dim, out_features=output_dim) #nn.Linearをself.lin_regとして定義。nn.Linearは全結合層（線形変換）を意味。
        
    def forward(self, x):   ## predicting functions（予測関数）
        out = self.lin_reg(x) #重みとバイアスを使って計算された出力
        return out

#モデルのインスタンス化 （モデルの実現化）
lin_reg = LinearRegression(input_size, output_size)  ## create an instance

## Training
print('Training started ____________________________________ \n')

#トレーニングの設定
## criteria
learning_rate = 0.001
n_epochs = 100

#損失関数と最適化  
loss = nn.MSELoss()   ## usin MSE
optimizer = torch.optim.SGD(lin_reg.parameters(), lr=learning_rate)

#トレーニングループ
for epoch in range(n_epochs):
    ## predicting
    y_pred_train = lin_reg(X_train_scaled)
    ## cal the loss
    l = loss(y_train, y_pred_train)
    ## backpropagation（勾配を計算する）
    l.backward()
    ## taka an optimization step
    optimizer.step()
    ## empty the gradients
    optimizer.zero_grad()
    
print('Training finished ____________________________________ \n') 
print()
mse_train = loss(y_pred_train, y_train)    
print('MSE train -->', mse_train.item())

[w, b] = lin_reg.parameters()
print('weight -->', w.detach().numpy())
print('bias -->', b.detach().numpy())

Training started ____________________________________ 

Training finished ____________________________________ 


MSE train --> 3198.805419921875
weight --> [[4.863918  8.30201   3.6997583 4.748607  3.5859802]]
bias --> [-0.18424499]


#### `Evaluation`

In [4]:
## you must stop gradient during inference to not kick the gradient graph during inference
#トレーニングおよびテスト後の評価
with torch.no_grad():
    y_pred_test = lin_reg(X_test_scaled)
    mse_test = loss(y_test, y_pred_test)
    print('MSE train -->', mse_test.item())

MSE train --> 2954.069580078125


#### Done!